In [1]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

api_key=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

/Users/mac/Documents/work/pinecone/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
data_job = {
    "id": 335465,
    "position": "APPLICATION DEVELOPER",
    "agency": "OFFICE OF EMERGENCY MANAGEMENT",
    "internal": True,
    "internal_id": 1,
    "title": "EMERGENCY PREPAREDNESS SPECIAL",
    "job_category": "Technology, Data & Innovation",
    "salary_range_from": 70000,
    "salary_range_to": 80000,
    "salary_frequency": "Annual",
    "work_location": "165 Cadman Plaza East",
    "work_location_address": "165 Cadman Plaza East Brooklyn, NY 11201",
    "work_location_city": "Brooklyn",
    "division": "Technology",
    "job_description": 'NYCEM has an exciting opportunity for a motivated scientific software development professional to join the Technology and Strategic Data team at New York City Emergency Management (NYCEM).   Under the supervision of the Director of Application Development, the selected applicant will be an Application Developer (EOC Application Developer) working on a portfolio of data management, business intelligence and situational awareness applications and will work with a team of Information Technology and Geographic Information Systems (GIS) application developers to maintain and enhance existing EOC systems. That team will also work on requirements gathering, developing new applications, supporting deployed systems, maintenance and enhancements of in-house and COTS products. The Application Developer will have to work on multiple projects with competing priorities. In addition, the Application Developer will:   √¢‚Ç¨¬¢	Gather requirements, document specifications and participate in agile development of information management software   √¢‚Ç¨¬¢	Integrate citywide data sources using a service-oriented approach to deliver real-time data for analysis and decision-making support √¢‚Ç¨¬¢	Provide research and advice on strategic software engineering and information management issues √¢‚Ç¨¬¢	Develop and maintain n-tier architecture applications for ingest, analysis, reporting, dissemination of data   √¢‚Ç¨¬¢	Build and maintain software for migration from legacy applications √¢‚Ç¨¬¢	Support MIS, applications development teams and agency users with software deployments √¢‚Ç¨¬¢	Accept ad hoc rapid application development requests, e.g, custom dashboards  √¢‚Ç¨¬¢	Support enterprise application deployments √¢‚Ç¨¬¢	Manage scripts, models, and tools production in an enterprise environment √¢‚Ç¨¬¢	Assist in the maintenance and updating of in-house EOC applications √¢‚Ç¨¬¢	Enhance data management capabilities via the automation of tasks and implementation of custom tools  The selected candidate will be assigned to periodic Emergency Operations Center team and will be expected to work non-business hours during emergencies. The selected candidate will also participate in drills and exercises, assist with Ready NY presentations to external groups, and will undertake special projects as assigned"	1. A master√¢‚Ç¨‚Ñ¢s degree from an accredited college in emergency management, public administration, urban planning, engineering, economics, political science, the physical sciences or related field and one year of satisfactory full-time professional experience in one or a combination of the following: emergency management, fire, police, or military service, public safety, public health, public administration, urban planning, engineering, or another specialized area to which the appointment is to be made; or  2. A baccalaureate degree from an accredited college and two years of satisfactory full-time professional experience in the areas listed in √¢‚Ç¨≈ì1√¢‚Ç¨¬ù above; or  3. A four-year high school diploma or its educational equivalent approved by a state√¢‚Ç¨‚Ñ¢s  department of education or a recognized accrediting organization and six years of satisfactory full-time professional experience in the areas listed in √¢‚Ç¨≈ì1√¢‚Ç¨¬ù above, at least two years of which must have been in one of those areas, or another specialized area to which the appointment is to be made.	"√¢‚Ç¨¬¢	Ability to write documented, maintainable and extensible code for a variety of application types. √¢‚Ç¨¬¢	Ability to serve as Technical Lead for a application development team. √¢‚Ç¨¬¢	Ability to perform code review with other developers. √¢‚Ç¨¬¢	Ability to perform QA testing. √¢‚Ç¨¬¢	Advanced knowledge of Visual Studio and source code version control. √¢‚Ç¨¬¢	Experience with JQueryUI and Telerik. √¢‚Ç¨¬¢	Experience with Microsoft .NET and C# and related technologies for web/desktop/mobile applications. √¢‚Ç¨¬¢	Significant development experience with MS SQL Server 2008 or later. √¢‚Ç¨¬¢	Experience with ASP.NET, WCF, WPF, ASP.NET MVC (3.0-5.0). √¢‚Ç¨¬¢	Experience with LINQ and Entity Framework. √¢‚Ç¨¬¢	Experience with NHibernate or another ORM. √¢‚Ç¨¬¢	Experience with CSS2, responsive design, and UI/UX engineering.  √¢‚Ç¨¬¢	Familiarity with geospatial data types. √¢‚Ç¨¬¢	Ability to thrive in agile software development approach √¢‚Ç¨¬¢	Demonstrated experience working with technical and non-technical staff. √¢‚Ç¨¬¢	Strong initiative and ability to perform with little supervision. √¢‚Ç¨¬¢	Ability to identify needs of end users and communicate possible solutions to them. √¢‚Ç¨¬¢	Strong organizational skills. √¢‚Ç¨¬¢	Ability to handle multiple projects simultaneously under tight deadlines. √¢‚Ç¨¬¢	Ability to work in a team environment."	THIS POSITION IS GRANT FUNDED THROUGH 8/31/2019 WITH THE POSSIBILITY OF AN EXTENSION	Current City Employees: Apply via Employee Self-Service (ESS).  Go to Recruiting Activities > Careers and search Job ID# 335465.  Non-City Employees/External Candidates: Apply via NYC Careers. Go to www.nyc.gov/careers and search Job ID# 335465.  NOTE: ONLY THOSE CANDIDATES UNDER CONSIDERATION WILL BE CONTACTED.	MONDAY-FRIDAY  9:00AM-5:00PM  The selected candidate will be assigned to periodic Emergency Operations Center team and will be expected to work non-business hours during emergencies. The selected candidate will also participate in drills and exercises, assist with Ready NY presentations to external groups, and will undertake special projects as assigned	165 Cadman Plaza East Brooklyn, NY 11201		New York City residency is generally required within 90 days of appointment. However, City Employees in certain titles who have worked for the City for 2 continuous years may also be eligible to reside in Nassau, Suffolk, Putnam, Westchester, Rockland, or Orange County. To determine if the residency requirement applies to you, please discuss with the agency representative at the time of interview'
}


In [3]:
index_name = "applicants"
index_resume = "applicants_resume"

if not pc.has_index(index_name):
    print("creating new index")

    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

creating new index


In [4]:
import time

while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

index = pc.Index(index_name)


{'ready': True, 'state': 'Ready'}